In [1]:
from sklearn import svm,datasets
iris=datasets.load_iris()

In [4]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df["flower"]=iris.target
df["flower"]=df["flower"].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=100)

In [11]:
model=svm.SVC(kernel="rbf",C=30,gamma="auto")
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9777777777777777

In [12]:
from sklearn.model_selection import cross_val_score


In [16]:
cross_val_score(svm.SVC(kernel="linear",C=30,gamma="auto"),iris.data,iris.target,cv=5)

array([1. , 1. , 0.9, 0.9, 1. ])

In [17]:
cross_val_score(svm.SVC(kernel="rbf",C=10,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [15]:
cross_val_score(svm.SVC(kernel="rbf",C=20,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

so every line we should change the value to get the good model,so we introduce a looping system but it is also not efficient number of variables depent the number of for loops so we use gridsearch

In [23]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(gamma="auto"),{
    "C":[1,10,20],
    "kernel":["rbf","linear"]
},cv=5,return_train_score=False)
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00160308, 0.0032052 , 0.        , 0.00168667, 0.00160165,
        0.        ]),
 'std_fit_time': array([0.00320616, 0.00392556, 0.        , 0.00337334, 0.0032033 ,
        0.        ]),
 'mean_score_time': array([0.00159335, 0.        , 0.00161886, 0.        , 0.        ,
        0.00160527]),
 'std_score_time': array([0.0031867 , 0.        , 0.00198269, 0.        , 0.        ,
        0.00321054]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [24]:
df=pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001603,0.003206,0.001593,0.003187,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.003205,0.003926,0.000000,0.000000,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000000,0.000000,0.001619,0.001983,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001687,0.003373,0.000000,0.000000,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001602,0.003203,0.000000,0.000000,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000000,0.000000,0.001605,0.003211,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [25]:
df[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [26]:
clf.best_score_

0.9800000000000001

In [27]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

if there are lot of datas and parameters computational power will increase

In [31]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(svm.SVC(gamma="auto"),{
    "C":[1,10,20],
    "kernel":["rbf","linear"]
},cv=5,return_train_score=False,n_iter=2)
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,linear,0.98
1,10,rbf,0.98


How to select the model

In [32]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [33]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [35]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=["model","best_score","best_params"])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
